# Homework 5

by Maia Rosengarten <br/>
SID: 23572580<br/>
March 27, 2017<br/>
Kaggle Score:<br/>

In [1]:
import matplotlib
import numpy as np
from numpy import linalg as LA
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy.stats import logistic as sig
import pandas as pd

np.set_printoptions(threshold=np.nan)
from scipy import io
import sklearn
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer as dv
from sklearn.preprocessing import Imputer as imp
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

# NP FNS

In [2]:
def vec(arr): # d b 1 --> (d, 1)
    return arr.reshape((arr.shape[0], 1))

In [3]:
def arr(vec): # 1 by d --> (d, )
    return vec.flatten()

# PREPROCESSING FNS

In [4]:
def vectorizeMatrix(data):
    '''
        Replaces all categorical variables that take on n possible values with
        n new columns in the design matrix. 
            e.g. If "Weather" takes on "sunny" or "rainy" would create two new features (columns)
                in the design matrix, one 'sunny' and the other 'rainy'. If sample point's weather is
                sunny, the new 'sunny' feature would take value 1, while 'rainy' would take 0.
        Args:
            data - a dataframe, the design matrix
    
    '''
    v = DictVectorizer(sparse=False)
    data = v.fit_transform(data.T.to_dict().values())
    return data

    

In [5]:
def fillMissingValuesWithAverage(designMatrix, isCat):
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        feature mean (if quantitative) and feature mode (if categorical)
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            isSCat (bool) - boolean set to True if feature is categorical, False if quantitative
    
    '''

In [6]:
def fillMissingValuesWithKNN(designMatrix, k):
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        value returned by K Nearest Neighbors
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            k (int) - determines the number of neighbors that will determine point's value
    
    '''


# UTIL FNS

In [7]:
def plotCostFN(costs):
    '''
        Plots cost function as a function of number of iterations
        Args:
            costs (ndarray) - lst of costs per iteration of gradient descent
    '''
   
    iters = [i for i in range(len(costs))]
    plt.plot(iters, costs)
    plt.xscale("log")
    plt.title("Cost Per Iteration of Gradient Descent")
    plt.xlabel('numIterations')
    plt.ylabel('cost')
    plt.show()

In [8]:
def generateCsv(aryPredictions, strCsvName):
    '''
    Writes predictions of testSet to csv file
    Args:
        aryPredictions (ndarray) - (nx1)-array of predictions given size n test (or valid) set
        strCsvName (str) - name of csv file to write to
    '''
    with open(strCsvName + '.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(aryPredictions)):
            writer.writerow([i, aryPredictions[i]])
    csvfile.close()

In [9]:
def readCSV(strFileName):
    '''
    Reads CSV file and returns data, labels tuple
    Args:
        strFileName (str) - path of csv file to read
    
    '''

In [10]:
def writeCleanedFeaturesToCSV(cleanedDesignMatrix):
    '''
        Writes cleaned version of design matrix (filled in missing values, processed categorical variables)
        to a CSV file to speed up work (don't need to preprocess every time you run code)
        
        Args: 
            cleanedDesignMatrix  
    
    '''

In [432]:
class DecisionTree:
    def __init__(self, root=None, max_depth=50, leaf_condition=1):
        self.root = root
        self.max_depth = max_depth
        self.leaf_condition = leaf_condition
    
    def train(self, train_data, train_labels):
        '''
            Sets the root of the decision tree to the root node of the resulting tree from buildTree.
            Args:
                train_data (ndarray) - cleaned training data
                train_lablels (ndarray) - cleaned training labels
                height_cap (int) - prevents tree from growing too large
        
        '''
        node_depth = 0
        self.root = self.buildTree(train_data, train_labels, self.max_depth, node_depth, self.leaf_condition)
    
    def buildTree(self, train_data, train_labels, max_depth, node_depth, leaf_condition=1):
        '''
            Recursively grows a decision tree by constructing nodes. Using the impurity and segmenter methods, 
            attempts to find a configuration of nodes that best splits the input data. 
            This function figures out the split rules that each node should have and figures out 
            when to stop growing the tree and insert a leaf node.  
            
            This phase is training the data.
            
            Args:
                train_data (ndarray) - n by d matrix of cleaned training data
                train_labels (ndarray) - n by 1 vector of training labels
                max_depth (int) - caps the height of the tree to prevent it from growing too large
                
        '''
        #fix this stopping condition
        numSamples = train_data.shape[0]
        numClass1 = np.sum(train_labels)
        numClass0 = numSamples - numClass1
        
#         #is leaf
        isLeaf, label = self.isMajClassLeaf(numSamples, numClass1, numClass0, leaf_condition)
        if (isLeaf):
            return Node(label=label)
        elif (max_depth==0): #isLeaf
            if (numClass1 > numClass0):
                label = 1
            else:
                label = 0
            return Node(label=label)  
        else:
            node = Node()
            node.chooseSplitRule(train_data, train_labels)
            
            leftIndices = [] 
            rightIndices = [] 
#             lstSortedTupIndex2FeatVal = [i for i in sorted(enumerate(train_data[:, node.splitFeatIndex]), key=lambda x: x[1])]
            
#             lstSortedIndices = [i[0] for i in lstSortedTupIndex2FeatVal]
#             lstSortedFeatVals = [i[1] for i in lstSortedTupIndex2FeatVal]

            for i in range(train_data.shape[0]):
                if (train_data[i, node.splitFeatIndex] <= node.splitVal):
                    leftIndices.append(i)
                else:
                    rightIndices.append(i)
            
            leftSetX = train_data[leftIndices, :]
            leftSetY = train_labels[leftIndices]

            
            rightSetX = train_data[rightIndices, :]
            rightSetY= train_labels[rightIndices]

            sizeRightSet = len(rightSetX)
            sizeLeftSet = len(leftSetX)
            
            if (sizeRightSet==0):
                if (np.sum(rightSetY) > sizeRightSet/2):
                    return Node(label=1)
                return Node(label=0)
            
            if (sizeLeftSet==0):
                if (np.sum(leftSetY)> sizeLeftSet/2):
                    return Node(label=1)
                return Node(label=0)
                
            max_depth -=1
            node_depth+=1
            return Node(featureIndex=node.splitFeatIndex, splitVal=node.splitVal, leftChild=self.buildTree(leftSetX, leftSetY, max_depth, node_depth, leaf_condition), rightChild=self.buildTree(rightSetX, rightSetY, max_depth, node_depth, leaf_condition), depth=node_depth)
    
        
    def predictPoint(self, node, data_point):
        '''
            Given a data point, traverse the tree to find the best label to classify the data point. 
            Start at the root node and evaluate split rules as you traverse until you reach a leaf. 
            Choose that leaf nodes label as your output label.
            
            data_point (ndarray) - (dx1) single point to classify with d features 
        
        '''
        if (node.label is not None):
            tortn = node.label
            return tortn  
        if (data_point[node.splitFeatIndex] <= node.splitVal):
            return self.predictPoint(node.left, data_point)
        else:
            return self.predictPoint(node.right, data_point)
    
    def predict(self, dataMatrix):
        '''
            Given a matrix of test samples, predict labels.
            
            dataMatrix (ndarray) - (nxd) unseen samples with d dimensions 
        ''' 
        predictions = []
        for i in range(dataMatrix.shape[0]):   
            label = self.predictPoint(self.root, dataMatrix[i])
            predictions.append(label)
        return predictions
    
    def isMajClassLeaf(self, numSamples, numClass1, numClass0, threshold):
        '''
            Returns (True, y) if threshold% of points in a node are of a particular class
            where y is the label of the leaf.

        '''     
        if (numClass1 >= (threshold * numSamples)):
            return True, 1
        elif (numClass0 >= (threshold * numSamples)):
            return True, 0
        return False, None

In [384]:
# def isPureLeaf(numSamples, numClass1, numClass0):
#     '''
#         Returns (True, y) if leaf is pure where y is the label of the leaf.
    
#     '''
#     if (numClass1 == numSamples): 
#         return True, 1
#     elif (numClass0 == numSamples):
#         return True, 0
#     return False, None

In [433]:
class RandomForest:
    def __init__(self, numTrees, train_data, train_labels, leaf_condition):
#         self.attributeBag = attributeBag #subset of features to train each node on in different trees
#         self.dataBag = dataBag #subset of data to train each tree on 
        self.train_data = train_data
        self.train_labels = train_labels
        self.trees = []
        self.predictions = []
        self.numTrees = numTrees
        self.leaf_condition = leaf_condition
        for i in range(self.numTrees):
            self.trees.append(DecisionTree(leaf_condition=self.leaf_condition))          
                     
    def train(self, train_data):
        i = 0
        for tree in self.trees:
            print('tree ' + str(i))
            numSamples = self.train_data.shape[0]
            random = np.random.choice(numSamples, numSamples, replace=True)
            subset_data = self.train_data[random]
            subset_labels = self.train_labels[random]
            tree.train(subset_data, subset_labels)
            i+=1
    
    def predict(self, valid_data):
        predictions = []
        for point in valid_data:
            labels = []
            for tree in self.trees:
                labels.append(tree.predictPoint(tree.root, point))
            if (np.sum(labels) >= 0.5 * self.numTrees):
                predictions.append(1)
            else:
                predictions.append(0)
        self.predictions = predictions

Go through all the spam data
Bag of words: have b words in vocabulary and vector is in R^b. vi = 1 if word i appears in document
if vocabulary is "ham", "spam", "sam" then you have your sentences: "sam likes spam": [0, 1, 1]...

When taking mutually exclusive sets, can overfit. Writing a better featurize.py
Take most commonly occurring words and make it an exclusive set. I get set of all words minus the stuff that the 
two classes have in common (like "the"). vocab of one class, vocab of two class. and can do set(v1) - set(v2) (but can overfit)...

if that can overfit, then can add other words from in between randomly. Or dimensionality reduction on total bag of words model.

In [438]:
class Node:
    def __init__(self, featureIndex=0, splitVal=None, leftChild=None, rightChild=None, label=None, depth=1):
        self.splitFeatIndex = featureIndex 
        self.splitVal = splitVal
        self.left = leftChild
        self.right = rightChild 
        self.label = label 
        self.depth = depth
    
    
    def chooseSplitRule(self, train_data, train_labels):
        '''
            Finds the best split rule for a Node using the impurity measure and input data. 
            Exhaustively tries threshold values from the data and chooses the combination of 
            split feature and threshold with the lowest impurity value. 
           
            Args:
               train_data (ndarray) - cleaned train data 
               train_labels (ndarray) - train labels
            
            Returns (bestSplitFeatIndex, bestSplitValue)
                 
            
            If xi is quantitative, sort points in S by feature xi; 
            remove duplicates; [duplicate values, not points]
            try splitting between each pair of consecutive values.
            [We can radix sort the values in linear time, and if n is huge we should.]
            
            Clever Bit: As you scan sorted list from left to right, 
            you can update entropy in O(1) time per point! 
            [This is important for obtaining a fast tree-building time.] <--GO OVER THIS.
            
            Need to deal with case when a child is empty (skip that split)
    
        '''
        #<= split value --> left, > split value --> right
        
        #H before (PARENT NODE)  
        currNodeSize = len(train_data)
        currClass1Count = sum(train_labels) 
        currClass0Count = currNodeSize - currClass1Count
        currNodeEntropy = self.computeEntropy((currNodeSize, currClass0Count, currClass1Count))
        bestInfoGain = -float("inf")
 
        #SET UP FOR SPLIT
        labelDictRight = {0: currClass0Count, 1: currClass1Count}
        labelDictLeft = {0:0, 1:0}
        sizeRightSet = currNodeSize
        sizeLeftSet = 0
        
        for featureIndex in range(train_data.shape[1]):
            sortedIndices = np.argsort(train_data[:, featureIndex])
            sortedVals = np.sort(train_data[:, featureIndex])
            
            lstUniqueVal, lstUniqueCounts = np.unique(sortedVals, return_counts=True)
            runningIndex = - 1
            for i in range(len(lstUniqueVal)):
                runningIndex += (lstUniqueCounts[i])
                tempSplitVal = lstUniqueVal[i]
                
                sizeLeftSet = runningIndex + 1
                sizeRightSet = currNodeSize - sizeLeftSet
                
                labelDictLeft[1] = np.sum(train_labels[sortedIndices[:runningIndex+1]])
                labelDictRight[1] = np.sum(train_labels[sortedIndices[runningIndex+1:]])
                labelDictRight[0] = sizeRightSet - labelDictRight[1]
                labelDictLeft[0] = sizeLeftSet - labelDictLeft[1]
                
                tupSizeLeftClassCounts = (sizeLeftSet, labelDictLeft[0], labelDictLeft[1])
                tupSizeRightClassCounts = (sizeRightSet, labelDictRight[0], labelDictRight[1])
                entropyAfterSplit = self.entropyAfterSplit(tupSizeLeftClassCounts, tupSizeRightClassCounts) 
                infoGain = currNodeEntropy - entropyAfterSplit
                
                if infoGain > bestInfoGain:
                    bestInfoGain = infoGain
                    bestSplitVal = tempSplitVal
                    bestSplitIndex = featureIndex 
  
        self.splitFeatIndex = bestSplitIndex 
        self.splitVal = bestSplitVal
              
    def entropyAfterSplit(self, tupSizeLeftClassCounts, tupSizeRightClassCounts):
        '''
            A method that takes in the result of a split: two histograms
            that count the frequencies of labels on the ”left” and ”right” side of that split.
            Calculates and outputs a scalar value representing the impurity (”badness”) of the 
            specified split on the input data. In lecture, this was refered to as H_after where H
            is the function that computes entropy.
            
            Args:
                tupSizeLeftClassCounts (tup) - 
                tupSizeRightClassCounts (tup) - 
        
        '''
        sizeLeftSet = tupSizeLeftClassCounts[0]
        sizeRightSet = tupSizeRightClassCounts[0]

        entropyLeft = self.computeEntropy(tupSizeLeftClassCounts)
        entropyRight = self.computeEntropy(tupSizeRightClassCounts)
        entropyAfterSplit = (entropyLeft * sizeLeftSet + entropyRight * sizeRightSet)/ (sizeLeftSet + sizeRightSet)
        return entropyAfterSplit
    
    
    def computeEntropy(self, tupleSizeSetClassCounts):
        '''
            Compute the entropy of a set (assume 2 classes). 
            
            Args:
                tupleSetSizeClassSize (tup) - (cardinality of set, cardinality of class 0 in set) 
        '''
        sizeSet, countClass0, countClass1 = tupleSizeSetClassCounts
        if (sizeSet==0):
            return 0
        probClass0 = countClass0/sizeSet
        if (probClass0 == 1 or probClass0 == 0):
            return 0
        else: 
            entropy = (- probClass0 * np.log2(probClass0)) - ((1 - probClass0) * np.log2(1 - probClass0))
        return entropy

# LOAD DATASET

In [166]:
#SPAM DATA

spamData = sp.io.loadmat("dist/spam_data.mat")
spamData.keys()
spamTrainX = spamData['training_data']
spamTrainY = spamData['training_labels']
spamTest = spamData['test_data']

combined = np.hstack([spamTrainX, spamTrainY.T])
np.random.shuffle(combined)
combined = imp().fit_transform(combined)

spamTrainX = combined[:, :-1]
spamTrainY = combined[:, -1]

#maybe add a bias feature
trainX, validX, trainY, validY = train_test_split(spamTrainX, spamTrainY, test_size=.1, random_state=42)
print("")

In [402]:
decisionTree = DecisionTree(max_depth=50, leaf_condition=.84)
decisionTree.train(trainX, trainY)

KeyboardInterrupt: 

In [351]:
predictions = decisionTree.predict(validX)
accuracy = accuracy_score(validY, predictions)

In [352]:
accuracy

0.78026149304091097

In [434]:
num_trees = 50
leaf_condition = 0.84
randForest = RandomForest(num_trees, trainX, trainY, leaf_condition)
randForest.train(trainX)
predictions = randForest.predict(validX)
# predictions

tree 0
tree 1
tree 2
tree 3
tree 4
tree 5
tree 6
tree 7
tree 8
tree 9
tree 10
tree 11
tree 12
tree 13
tree 14
tree 15
tree 16
tree 17
tree 18
tree 19
tree 20
tree 21
tree 22
tree 23
tree 24
tree 25
tree 26
tree 27
tree 28
tree 29
tree 30
tree 31
tree 32
tree 33
tree 34
tree 35
tree 36
tree 37
tree 38
tree 39
tree 40
tree 41
tree 42
tree 43
tree 44
tree 45
tree 46
tree 47
tree 48
tree 49


In [435]:
randForest.predictions is None

False

In [436]:
accuracy = accuracy_score(validY, randForest.predictions)

In [437]:
accuracy

0.77815267819485445